In [7]:
# Import necessary libraries
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image
from time import sleep
import os
import numpy as np
from sklearn.metrics import r2_score
import sklearn as sk
import math
from amb_sdk.sdk import DarwinSdk

ds = DarwinSdk()
ds.set_url('https://amb-demo-api.sparkcognition.com/v1/')
status, msg = ds.auth_login_user('EM.J.REINHART@GMAIL.COM', 'xeyW2ue46p')

if not status:
    print(msg)

In [8]:
# Preview dataset
df = pd.read_csv("austin-animal-center-outcomes-1.csv")
df.head(10)

# How many NaN values in Outcome Subtype
# newData = data.fillna("")
# subtype = newData.get('Outcome Subtype')
# empty = 0
# for entry in subtype:
#     if entry== "":
#         empty += 1
# print(empty)

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A771830,*Bradley,12/27/2018 01:25:00 PM,12/27/2018 01:25:00 PM,05/10/2017,Adoption,Foster,Dog,Neutered Male,1 year,Pit Bull Mix,White/Blue
1,A779576,*Rajah,10/01/2018 05:03:00 PM,10/01/2018 05:03:00 PM,04/18/2018,Adoption,NaN,Cat,Neutered Male,5 months,Domestic Shorthair Mix,Brown Tabby
2,A741715,*Pebbles,01/11/2017 06:17:00 PM,01/11/2017 06:17:00 PM,03/07/2016,Adoption,NaN,Cat,Spayed Female,10 months,Domestic Shorthair Mix,Calico
3,A658751,Benji,11/13/2016 01:38:00 PM,11/13/2016 01:38:00 PM,07/14/2011,Return to Owner,NaN,Dog,Neutered Male,5 years,Border Terrier Mix,Tan
4,A721285,NaN,02/24/2016 02:42:00 PM,02/24/2016 02:42:00 PM,02/24/2014,Euthanasia,Suffering,Other,Unknown,2 years,Raccoon Mix,Black/Gray
5,A746650,Rose,04/07/2017 11:58:00 AM,04/07/2017 11:58:00 AM,04/06/2016,Return to Owner,NaN,Dog,Intact Female,1 year,Labrador Retriever/Jack Russell Terrier,Yellow
6,A750122,Happy Camper,05/24/2017 06:36:00 PM,05/24/2017 06:36:00 PM,04/08/2017,Transfer,Partner,Dog,Intact Male,1 month,Labrador Retriever Mix,Black
7,A756696,Shakti,09/01/2017 11:23:00 AM,09/01/2017 11:23:00 AM,08/24/2014,Return to Owner,NaN,Cat,Spayed Female,3 years,Domestic Shorthair Mix,Blue/White
8,A784168,NaN,11/14/2018 12:33:00 PM,11/14/2018 12:33:00 PM,03/12/2018,Transfer,Partner,Dog,Intact Male,8 months,English Springer Spaniel Mix,Black/White
9,A707443,NaN,07/13/2015 01:50:00 PM,07/13/2015 01:50:00 PM,06/21/2015,Transfer,Partner,Cat,Intact Female,3 weeks,Domestic Longhair Mix,Black Smoke


In [9]:
# remove unwanted columns
# remove Name column
noNames = df.drop(['Name'], axis = 1)
noNames.head()
# remove MonthYear column
oneDate = noNames.drop(['MonthYear'], axis = 1)
oneDate.head()
# remove Outcome Subtype
noSubtype = oneDate.drop(['Outcome Subtype'], axis = 1)
noSubtype.head()
cleanData = noSubtype

In [10]:
# feature engineer DateTime and Date of Birth to get age upon arrival
from datetime import datetime
def getDays(entry):
    splitEntry = entry.split(" ")
    dates = splitEntry[0].split("/")
    month = dates[0]
    month = int(month)
    day = dates[1]
    day = int(day)
    year = dates[2]
    year = int(year)
    numDays = datetime(year, month, day, 0, 0, 0)
    return numDays


#birthday = datetime(year, month, day, hour, min, sec)
#dateEntry = datetime()
#diff = dateEntry - birthday
#prints (days)

In [11]:
ageMonths = {}   
for index, row in cleanData.iterrows():
    entryDate = row['DateTime']
    entryDays = getDays(entryDate)
    birthday = row['Date of Birth']
    numBirth = getDays(birthday)
    diff = entryDays - numBirth
    ageMonths[entryDate] = diff
    
animalData = cleanData.copy()
animalData['Age Upon Arrival'] = cleanData['DateTime'].map(ageMonths)
animalData.head()

,Animal ID,DateTime,Date of Birth,Outcome Type,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color,Age Upon Arrival
0,A771830,12/27/2018 01:25:00 PM,05/10/2017,Adoption,Dog,Neutered Male,1 year,Pit Bull Mix,White/Blue,596 days
1,A779576,10/01/2018 05:03:00 PM,04/18/2018,Adoption,Cat,Neutered Male,5 months,Domestic Shorthair Mix,Brown Tabby,68 days
2,A741715,01/11/2017 06:17:00 PM,03/07/2016,Adoption,Cat,Spayed Female,10 months,Domestic Shorthair Mix,Calico,310 days
3,A658751,11/13/2016 01:38:00 PM,07/14/2011,Return to Owner,Dog,Neutered Male,5 years,Border Terrier Mix,Tan,1949 days
4,A721285,02/24/2016 02:42:00 PM,02/24/2014,Euthanasia,Other,Unknown,2 years,Raccoon Mix,Black/Gray,730 days


In [24]:
animalData.to_csv('animalDataCSV2.csv', sep=',')
animalData.to_json("datasetconverted.json", orient='index')

In [25]:
# upload
status, dataset = ds.upload_dataset("animalDataCSV2.csv")
if not status:
    print(dataset)

In [27]:
# clean dataset
target = "Outcome Type"
status, job_id = ds.clean_data("animalDataCSV2.csv", target = target, index = "DateTime")

if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

{'status': 'Requested', 'starttime': '2019-04-11T16:16:33.021937', 'endtime': None, 'percent_complete': 0, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['animalDataCSV2.csv'], 'artifact_names': ['8a4746bc85ef4e4d8cf2f670308846d1'], 'model_name': None, 'job_error': None}
{'status': 'Running', 'starttime': '2019-04-11T16:16:33.021937', 'endtime': None, 'percent_complete': 0, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['animalDataCSV2.csv'], 'artifact_names': ['8a4746bc85ef4e4d8cf2f670308846d1'], 'model_name': None, 'job_error': ''}
{'status': 'Running', 'starttime': '2019-04-11T16:16:33.021937', 'endtime': None, 'percent_complete': 0, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['animalDataCSV2.csv'], 'artifact_names': ['8a4746bc85ef4e4d8cf2f670308846d1'], 'model_name': None, 'job_error': ''}
{'status': 'Running', 'starttime': '2019-04-11T16:16:33.021937', 'endtime': None, 'perc

In [28]:
dataset_name = "animalDataCSV2.csv"
model = target + "_model0"
status, job_id = ds.create_model(dataset_names = dataset_name, \
                                 model_name =  model, \
                                 max_train_time = '00:02')
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

{'status': 'Requested', 'starttime': '2019-04-11T16:18:05.997083', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['animalDataCSV2.csv'], 'artifact_names': None, 'model_name': 'Outcome Type_model0', 'job_error': None}
{'status': 'Running', 'starttime': '2019-04-11T16:18:05.997083', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['animalDataCSV2.csv'], 'artifact_names': None, 'model_name': 'Outcome Type_model0', 'job_error': ''}
{'status': 'Running', 'starttime': '2019-04-11T16:18:05.997083', 'endtime': None, 'percent_complete': 10, 'job_type': 'TrainModel', 'loss': 2.0147063732147217, 'generations': 0, 'dataset_names': ['animalDataCSV2.csv'], 'artifact_names': None, 'model_name': 'Outcome Type_model0', 'job_error': ''}
{'status': 'Running', 'starttime': '2019-04-11T16:18:05.997083', 'endtime': None, 'percent_complete': 10, 'job_type': 'TrainModel', 'lo

In [31]:
# Train some more
status, job_id = ds.resume_training_model(dataset_names = dataset_name,
                                          model_name = model,
                                          max_train_time = '00:05')
                                          
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

{'status': 'Requested', 'starttime': '2019-04-11T16:27:42.450509', 'endtime': None, 'percent_complete': 0, 'job_type': 'UpdateModel', 'loss': 1.796737790107727, 'generations': 1, 'dataset_names': ['animalDataCSV2.csv'], 'artifact_names': None, 'model_name': 'Outcome Type_model0', 'job_error': None}
{'status': 'Running', 'starttime': '2019-04-11T16:27:42.450509', 'endtime': None, 'percent_complete': 0, 'job_type': 'UpdateModel', 'loss': 1.796737790107727, 'generations': 1, 'dataset_names': ['animalDataCSV2.csv'], 'artifact_names': None, 'model_name': 'Outcome Type_model0', 'job_error': ''}
{'status': 'Running', 'starttime': '2019-04-11T16:27:42.450509', 'endtime': None, 'percent_complete': 0, 'job_type': 'UpdateModel', 'loss': 1.796737790107727, 'generations': 1, 'dataset_names': ['animalDataCSV2.csv'], 'artifact_names': None, 'model_name': 'Outcome Type_model0', 'job_error': ''}
{'status': 'Running', 'starttime': '2019-04-11T16:27:42.450509', 'endtime': None, 'percent_complete': 8, 'jo

ConnectionError: HTTPSConnectionPool(host='amb-demo-api.sparkcognition.com', port=443): Max retries exceeded with url: /v1/auth/login/user (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000026CB2537048>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [29]:
# Retrieve feature importance of built model
status, artifact = ds.analyze_model(model)
sleep(1)
if status:
    ds.wait_for_job(artifact['job_name'])
else:
    print(artifact)
status, feature_importance = ds.download_artifact(artifact['artifact_name'])

{'status': 'Running', 'starttime': '2019-04-11T16:22:04.386643', 'endtime': None, 'percent_complete': 0, 'job_type': 'AnalyzeModel', 'loss': 1.796737790107727, 'generations': 1, 'dataset_names': None, 'artifact_names': ['2a49e8899206466e89d0f44fb935e8d0'], 'model_name': 'Outcome Type_model0', 'job_error': ''}
{'status': 'Complete', 'starttime': '2019-04-11T16:22:04.386643', 'endtime': '2019-04-11T16:22:10.681458', 'percent_complete': 100, 'job_type': 'AnalyzeModel', 'loss': 1.796737790107727, 'generations': 1, 'dataset_names': None, 'artifact_names': ['2a49e8899206466e89d0f44fb935e8d0'], 'model_name': 'Outcome Type_model0', 'job_error': ''}


In [30]:
feature_importance[:10]

Unnamed: 0                          0.234842
Sex upon Outcome = Spayed Female    0.099306
Sex upon Outcome = Neutered Male    0.092851
Animal Type = Cat                   0.081702
Animal Type = Dog                   0.062188
Animal Type = Other                 0.045784
Age upon Outcome = 2 months         0.045326
Sex upon Outcome = Intact Male      0.040371
Sex upon Outcome = Unknown          0.036312
Age upon Outcome = 1 year           0.020031
dtype: float64